In [2]:
import os

In [1]:
%pwd

'd:\\aiadventures\\Study_Material\\DS_DEPLOYMENT_with_MLflow\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\aiadventures\\Study_Material\\DS_DEPLOYMENT_with_MLflow'

In [40]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL:str
    local_data_file:Path
    unzip_dir: Path


In [41]:
from end_2_end_ds_project.constants import *
from end_2_end_ds_project.utils.common import read_yaml, create_directories

In [42]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath  = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])



    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        
        return data_ingestion_config

In [48]:
import os
import urllib.request as request
import zipfile
from end_2_end_ds_project import logger
from end_2_end_ds_project.utils.common import get_size

In [56]:
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config  = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with the following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok = True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)



In [57]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e


[2024-12-23 17:15:19,334: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-23 17:15:19,338: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-23 17:15:19,342: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-12-23 17:15:19,346: INFO: common: created directory at: artifacts]
[2024-12-23 17:15:19,349: INFO: common: created directory at: artifacts/data_ingestion]
[2024-12-23 17:15:22,305: INFO: 2968097854: artifacts/data_ingestion/data.zip download! with the following info: 
Connection: close
Content-Length: 11824
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dc1778d5e1ea15ce79e64edf1947d90908450943a5163b1655ccc5b89d67ee9d"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 881F:DEA0D:6E7225:913A7A:67694D50
Accept-Ranges: bytes
Date: Mon,